In [1]:
import sys
sys.executable

'/home/rs37890/.conda/envs/UDA/bin/python'

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display as ipy_display, clear_output

In [4]:
deplabv3 = __import__('Deeplabv3')
import networks

In [5]:
dataset = __import__('dataset-inference')

# parameter

In [6]:
suffix = '130000'
num_classes = 5
batch_size = 1

In [23]:
label_ids = {"ignore": 0,
    "lv_myo": 1,
    "la_blood": 2,
    "lv_blood": 3,
    "aa": 4,
}

# path

In [7]:
root = './../data/'

In [8]:
target_ct_train_dir = root + "h5py/"
target_ct_test_dir = root + "h5py/"

# cuda

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#  sliding_window

In [10]:
def sliding_window(input_volume, window_size=(32, 32, 32), stride=(16, 16, 16)):
    
    z_max = input_volume.shape[0] - window_size[0] + 1
    x_max = input_volume.shape[1] - window_size[1] + 1
    y_max = input_volume.shape[2] - window_size[2] + 1

    windows = []

    for y in range(0, y_max, stride[2]):
        for x in range(0, x_max, stride[1]):
            for z in range(0, z_max, stride[0]):
                window = input_volume[z:z+window_size[0], x:x+window_size[1], y:y+window_size[2]]
                windows.append(window)

            # z_remaining
            z_remaining = input_volume.shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
            window = input_volume[z_remaining:, x:x+window_size[1], y:y+window_size[2]]
            windows.append(window)
        
        # x_remaining
        x_remaining = input_volume.shape[1] - window_size[1] # z_remaining = 78 - 32 = 46
        for z in range(0, z_max, stride[0]):
            window = input_volume[z:z+window_size[0], x_remaining: , y:y+window_size[2]]
            windows.append(window)
            
        # x_remaining z_remaining
        z_remaining = input_volume.shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
        window = input_volume[z_remaining:, x_remaining: , y:y+window_size[2]]
        windows.append(window)
    
    # y_remaining
    y_remaining = input_volume.shape[2] - window_size[2] # z_remaining = 78 - 32 = 46
    for x in range(0, x_max, stride[1]):
        for z in range(0, z_max, stride[0]):
            window = input_volume[z:z+window_size[0], x:x+window_size[1], y_remaining: ]
            windows.append(window)
            
        # y_remaining z_remaining
        z_remaining = input_volume.shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
        window = input_volume[z_remaining:, x:x+window_size[1], y_remaining:]
        windows.append(window)

    # y_remaining x_remaining
    x_remaining = input_volume.shape[1] - window_size[1] # z_remaining = 78 - 32 = 46
    for z in range(0, z_max, stride[0]):
        window = input_volume[z:z+window_size[0], x_remaining: , y_remaining:]
        windows.append(window)

    # y_remaining x_remaining z_remaining
    z_remaining = input_volume.shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
    window = input_volume[z_remaining:, x_remaining: , y_remaining:]
    windows.append(window)

    return windows

In [11]:
def combine_windows(window_outputs, input_volume_shape, window_size=(32, 32, 32), stride=(16, 16, 16)):
    num_classes = window_outputs[0].shape[1] # 5
    combined_prob = torch.zeros((num_classes,) + input_volume_shape).to(device)
    count_matrix = torch.zeros(input_volume_shape).to(device)

    z_max = input_volume_shape[0] - window_size[0] + 1
    x_max = input_volume_shape[1] - window_size[1] + 1
    y_max = input_volume_shape[2] - window_size[2] + 1

    idx = 0
    
    
    for y in range(0, y_max, stride[2]):
        for x in range(0, x_max, stride[1]):
            for z in range(0, z_max, stride[0]):
                output = window_outputs[idx].squeeze() # output.cpu().numpy().shape: (5, 32, 256, 256)
                combined_prob[:, z:z+window_size[0], x:x+window_size[1], y:y+window_size[2]] += output
                count_matrix[z:z+window_size[0], x:x+window_size[1], y:y+window_size[2]] += 1
                idx += 1
                

            # z_remaining
            z_remaining = input_volume_shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
            output = window_outputs[idx].squeeze()
            combined_prob[:, z_remaining:, x:x+window_size[1], y:y+window_size[2]] += output
            count_matrix[z_remaining:, x:x+window_size[1], y:y+window_size[2]] += 1
            idx += 1
        
        # x_remaining
        x_remaining = input_volume_shape[1] - window_size[1] # z_remaining = 78 - 32 = 46
        for z in range(0, z_max, stride[0]):
            output = window_outputs[idx].squeeze()
            combined_prob[:, z:z+window_size[0], x_remaining: , y:y+window_size[2]] += output
            count_matrix[z:z+window_size[0], x_remaining: , y:y+window_size[2]] += 1
            idx += 1
            
            
        # x_remaining z_remaining
        z_remaining = input_volume_shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
        output = window_outputs[idx].squeeze()
        combined_prob[:, z_remaining:, x_remaining: , y:y+window_size[2]] += output
        count_matrix[z_remaining:, x_remaining: , y:y+window_size[2]] += 1
        idx += 1
        
    
    # y_remaining
    y_remaining = input_volume_shape[2] - window_size[2] # z_remaining = 78 - 32 = 46
    for x in range(0, x_max, stride[1]):
        for z in range(0, z_max, stride[0]):
            output = window_outputs[idx].squeeze()
            combined_prob[:, z:z+window_size[0], x:x+window_size[1], y_remaining: ] += output
            count_matrix[z:z+window_size[0], x:x+window_size[1], y_remaining: ] += 1
            idx += 1
            
            
        # y_remaining z_remaining
        z_remaining = input_volume_shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
        output = window_outputs[idx].squeeze()
        combined_prob[:, z_remaining:, x:x+window_size[1], y_remaining:] += output
        count_matrix[z_remaining:, x:x+window_size[1], y_remaining:] += 1
        idx += 1
        

    # y_remaining x_remaining
    x_remaining = input_volume_shape[1] - window_size[1] # z_remaining = 78 - 32 = 46
    for z in range(0, z_max, stride[0]):
        output = window_outputs[idx].squeeze()
        combined_prob[:, z:z+window_size[0], x_remaining: , y_remaining:] += output
        count_matrix[z:z+window_size[0], x_remaining: , y_remaining:] += 1
        idx += 1

    # y_remaining x_remaining z_remaining
    z_remaining = input_volume_shape[0] - window_size[0] # z_remaining = 78 - 32 = 46
    output = window_outputs[idx].squeeze()
    combined_prob[:, z_remaining:, x_remaining: , y_remaining:] += output
    count_matrix[z_remaining:, x_remaining: , y_remaining:] += 1
    idx += 1
    
    
    # Normalize the class probabilities
    combined_prob /= count_matrix

    # Take the argmax of the accumulated probabilities
    combined_output = torch.argmax(combined_prob, dim=0)

    return combined_output

# Load dataset

In [12]:
dataloader = dataset.get_dataloader( target_ct_train_dir,  target_ct_test_dir, num_classes, batch_size,  domain = 'target' )

train_dataset = dataloader["train"].dataset
test_dataset = dataloader["test"].dataset

# Load checkpoint

In [13]:
dpv3 = deplabv3.DeepLabV3(num_classes)
classifier = networks.classifier(num_classes)

dpv3 = dpv3.to(device)
classifier = classifier.to(device)

# parallel
dpv3 = torch.nn.DataParallel(dpv3)
classifier = torch.nn.DataParallel(classifier)

In [14]:
dpv3_checkpoint = torch.load('./record-data/' + 'dpv3_weights_' + suffix + '.pth')
classifier_checkpoint = torch.load('./record-data/' + 'classifier_weights_' + suffix + '.pth')

dpv3.load_state_dict(dpv3_checkpoint)
classifier.load_state_dict(classifier_checkpoint)
print("Loaded model weights")

Loaded model weights


# iterate through test dataset

In [15]:
test_output = []

for img_idx in range(len(test_dataset)): # 0, 1, 2, 3
    
    data_vol, label_vol = test_dataset[img_idx] # data_vol: torch.Size([1, 60, 512, 512])
    data_vol = data_vol.to(device)
    label_vol = label_vol.to(device)
    
    data_vol = torch.squeeze(data_vol, 0) # data_vol:  torch.Size([60, 512, 512])
    windows = sliding_window(data_vol) # slice 3D image based on window size and stride
    
    
    
    window_outputs = []
    
    dpv3.eval()
    classifier.eval() 
    with torch.no_grad():
        for window in windows:
            window = window.unsqueeze(0)  # Add a channel dimension: torch.Size([1, 32, 256, 256])
            window = torch.unsqueeze(window, 0)  # Add a batch dimension: torch.Size([1, 1, 32, 256, 256])
            
            # inference
            output = dpv3(window)
            output = classifier(output) # torch.Size([1, 5, 32, 256, 256])
            output = F.softmax(output, dim=1)
            # collect outputs
            window_outputs.append(output)  # len(window_outputs) = 27
            # window_outputs[0].cpu().numpy().shape： (1, 5, 32, 256, 256)

    combined_output = combine_windows(window_outputs, data_vol.size())
    test_output.append(combined_output)

/home/rs37890/.conda/envs/UDA/lib/python3.9/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1712608885084/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(


In [16]:
len(window_outputs)

270

# evaluate

In [17]:
numpy_arrays = [tensor.cpu().numpy() for tensor in test_output]

In [18]:
numpy_arrays[0].shape

(149, 113, 123)

In [19]:
numpy_arrays[1].shape

(215, 123, 85)

In [20]:
numpy_arrays[2].shape

(258, 115, 52)

In [21]:
numpy_arrays[3].shape

(148, 107, 94)

In [26]:
id_to_ignore = 0
intersection = dict()
total = dict()
for label in label_ids:
    intersection[label] = total[label] = 0


for img_idx in range(len(test_dataset)): # 0, 1, 2, 3
    
    
    print('img_idx: ', )
    
    _, y_true = test_dataset[img_idx] # data_vol: torch.Size([1, 60, 512, 512])
    
    y_hat = numpy_arrays[img_idx]
    y_true = y_true.cpu().numpy() 
    
    print(y_hat.shape)
    print(y_true.shape)
    
    for label in label_ids:
        if label_ids[label] == id_to_ignore:
            continue

        curr_id = label_ids[label]

        idx_gt = y_true == curr_id
        idx_hat = y_hat == curr_id

        intersection[label] += 2 * np.sum(idx_gt & idx_hat)
        total[label] += np.sum(idx_gt) + np.sum(idx_hat)
        
dice = []
res = dict()
for label in label_ids:
    if label_ids[label] == id_to_ignore:
        continue
        
    if total[label] != 0:
        res[label] = intersection[label] / total[label]
    else:
        print('total is zero')
        res[label] = np.float64(0)

    dice.append(res[label])

img_idx: 
(149, 113, 123)
(149, 113, 123)
img_idx: 
(215, 123, 85)
(215, 123, 85)
img_idx: 
(258, 115, 52)
(258, 115, 52)
img_idx: 
(148, 107, 94)
(148, 107, 94)


In [27]:
np.mean(dice)
# train: 0.3411974349989537

0.3750375026439753

In [28]:
for k in res:
    print(k, res[k])

lv_myo 0.10943875175465523
la_blood 0.32024603684052294
lv_blood 0.5778663767065362
aa 0.4925988452741867


In [30]:
id_to_ignore = 0
for img_idx in range(len(test_dataset)): # 0, 1, 2, 3
    
    intersection = dict()
    total = dict()
    for label in label_ids:
        intersection[label] = total[label] = 0
        
    print('img_idx: ', img_idx)
    
    _, y_true = test_dataset[img_idx] # data_vol: torch.Size([1, 60, 512, 512])
    
    y_hat = numpy_arrays[img_idx]
    y_true = y_true.cpu().numpy() 
    
    print(y_hat.shape)
    print(y_true.shape)
    
    for label in label_ids:
        if label_ids[label] == id_to_ignore:
            continue

        curr_id = label_ids[label]

        idx_gt = y_true == curr_id
        idx_hat = y_hat == curr_id

        intersection[label] += 2 * np.sum(idx_gt & idx_hat)
        total[label] += np.sum(idx_gt) + np.sum(idx_hat)
        
    dice = []
    res = dict()
    for label in label_ids:
        if label_ids[label] == id_to_ignore:
            continue
            
        if total[label] != 0:
            res[label] = intersection[label] / total[label]
        else:
            print('total is zero')
            res[label] = np.float64(0)
    
        dice.append(res[label])
        
        
    for k in res:
        print(k, res[k])
    
    print('mean: ', np.mean(dice))
    print('-'*26)

img_idx:  0
(149, 113, 123)
(149, 113, 123)
lv_myo 0.17188946185376172
la_blood 0.6089401949149494
lv_blood 0.7321012162728925
aa 0.26732447540968657
mean:  0.4450638371128226
--------------------------
img_idx:  1
(215, 123, 85)
(215, 123, 85)
lv_myo 0.04942257867121486
la_blood 0.2141980441065843
lv_blood 0.5161452394124487
aa 0.5898659767649395
mean:  0.34240795973879684
--------------------------
img_idx:  2
(258, 115, 52)
(258, 115, 52)
lv_myo 0.0030187513601932
la_blood 0.2494498577173387
lv_blood 0.18847719467638055
aa 0.5956468181169343
mean:  0.2591481554677117
--------------------------
img_idx:  3
(148, 107, 94)
(148, 107, 94)
lv_myo 0.2251091947586516
la_blood 0.05663522117403914
lv_blood 0.7044599424870327
aa 0.24433529183452227
mean:  0.3076349125635614
--------------------------
